In [24]:
from datasets import load_dataset# comes with distilabel

# Using OPENAI API and gpt-3.5-turbo as LLM

In [12]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY']= userdata.get('OPENAI_API_KEY')

In [27]:
import openai
response = openai.models.list()
print(f"✅ API key is valid! Available models:{response}")

✅ API key is valid! Available models:SyncPage[Model](data=[Model(id='gpt-4o-audio-preview-2024-12-17', created=1734034239, object='model', owned_by='system'), Model(id='dall-e-3', created=1698785189, object='model', owned_by='system'), Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'), Model(id='gpt-4o-audio-preview-2024-10-01', created=1727389042, object='model', owned_by='system'), Model(id='o1-2024-12-17', created=1734326976, object='model', owned_by='system'), Model(id='o1', created=1734375816, object='model', owned_by='system'), Model(id='gpt-4o-realtime-preview-2024-10-01', created=1727131766, object='model', owned_by='system'), Model(id='gpt-4o-realtime-preview', created=1727659998, object='model', owned_by='system'), Model(id='babbage-002', created=1692634615, object='model', owned_by='system'), Model(id='o3-mini-2025-01-31', created=1738010200, object='model', owned_by='system'), Model(id='tts-1-hd-1106', created=1699053533, object='model', owned_by='

### loading Dataset 1 ->  better together dataset(huggingface & Argilla collab)

In [5]:
dataset = load_dataset("DIBT/10k_prompts_ranked")
dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/11.4k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.58M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10331 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'quality', 'metadata', 'avg_rating', 'num_responses', 'agreement_ratio', 'raw_responses', 'kind', 'cluster_description', 'topic'],
        num_rows: 10331
    })
})

In [6]:
dataset["train"][0]

{'prompt': 'Provide step-by-step instructions on how to make a safe and effective homemade all-purpose cleaner from common household ingredients. The guide should include measurements, tips for storing the cleaner, and additional variations or scents that can be added. Additionally, the guide should be written in clear and concise language, with helpful visuals or photographs to aid in the process.',
 'quality': [{'user_id': 'd23b12c2-b601-490e-b5b3-2040eb393a00',
   'value': '4',
   'status': 'submitted'},
  {'user_id': 'e2bdd868-f28e-46fc-9254-a6ec1e291889',
   'value': '4',
   'status': 'submitted'}],
 'metadata': '{"source": "ultrachat", "kind": "synthetic", "evolved_from": null}',
 'avg_rating': 5.0,
 'num_responses': 2,
 'agreement_ratio': 1.0,
 'raw_responses': [5, 5],
 'kind': 'synthetic',
 'cluster_description': 'Sustainable Packaging & Skin Care Products',
 'topic': 'Environmental Issues'}

###Loading another dataset with math topics as list of dictionaries

In [42]:

math_topics = [
    "Algebraic Expressions",
    "Linear Equations",
    "Quadratic Equations",
    "Polynomial Functions",
    "Rational Expressions",
    "Exponential Functions",
    "Logarithmic Functions",
    "Sequences and Series",
    "Matrices",
    "Determinants",
    "Complex Numbers",
    "Trigonometry",
    "Geometry",
    "Coordinate Geometry",
    "Vector Algebra",
    "Statistics",
    "Probability",
    "Calculus",
    "Differential Calculus",
    "Integral Calculus",
    "Limits and Continuity",
    "Differentiation",
    "Integration",
    "Theorems of Calculus",
    "Mathematical Reasoning",
    "Set Theory",
    "Number Theory",
    "Permutations and Combinations",
    "Binomial Theorem",
    "Arithmetic Progressions",
    "Geometric Progressions",
    "Harmonic Progressions",
    "Trigonometric Ratios",
    "Trigonometric Identities",
    "Inverse Trigonometric Functions",
    "Hyperbolic Functions",
    "Conic Sections",
    "Circle Geometry",
    "Ellipse Geometry",
    "Parabola Geometry",
    "Hyperbola Geometry",
    "Function Theory",
    "Graph Theory",
    "Differential Equations",
    "Mathematical Induction",
    "Discrete Mathematics",
]

dataset2 = [{"topic": topic} for topic in math_topics]
print(dataset2)


[{'topic': 'Algebraic Expressions'}, {'topic': 'Linear Equations'}, {'topic': 'Quadratic Equations'}, {'topic': 'Polynomial Functions'}, {'topic': 'Rational Expressions'}, {'topic': 'Exponential Functions'}, {'topic': 'Logarithmic Functions'}, {'topic': 'Sequences and Series'}, {'topic': 'Matrices'}, {'topic': 'Determinants'}, {'topic': 'Complex Numbers'}, {'topic': 'Trigonometry'}, {'topic': 'Geometry'}, {'topic': 'Coordinate Geometry'}, {'topic': 'Vector Algebra'}, {'topic': 'Statistics'}, {'topic': 'Probability'}, {'topic': 'Calculus'}, {'topic': 'Differential Calculus'}, {'topic': 'Integral Calculus'}, {'topic': 'Limits and Continuity'}, {'topic': 'Differentiation'}, {'topic': 'Integration'}, {'topic': 'Theorems of Calculus'}, {'topic': 'Mathematical Reasoning'}, {'topic': 'Set Theory'}, {'topic': 'Number Theory'}, {'topic': 'Permutations and Combinations'}, {'topic': 'Binomial Theorem'}, {'topic': 'Arithmetic Progressions'}, {'topic': 'Geometric Progressions'}, {'topic': 'Harmonic

In [15]:
from distilabel.models import OpenAILLM

llm = OpenAILLM(model="gpt-3.5-turbo")

#Text Generation

###Given dataset1 do text generations for instructions

In [32]:
from distilabel.pipeline import Pipeline
from distilabel.steps import LoadDataFromHub, KeepColumns
from distilabel.steps.tasks import TextGeneration
#With dataset1
with Pipeline(name="text_generation_dataset1") as pipeline_text_generation1:
    load_hub_dataset = LoadDataFromHub(
        name="load_dataset",
        repo_id="DIBT/10k_prompts_ranked",
        output_mappings={"prompt": "instruction"},
        split="train",
        num_examples=1
    )
    text_generation = TextGeneration(name="text_generation", llm=llm)
    keep_columns = KeepColumns(
        name="keep_columns",
        columns=[
            "instruction",
            "generation",
        ],
    )
    load_hub_dataset >> text_generation >> keep_columns


distiset_text_generation1 = pipeline_text_generation1.run(use_cache=False)


[04/14/25 10:04:31] INFO     ['distilabel.pipeline'] 📝 Pipeline data will be written to               ]8;id=959544;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=81408;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py#1015\1015]8;;\
                             '/root/.cache/distilabel/pipelines/text_generation_dataset1/e1db5331241e3             
                             7e8f4757d987fe77b7902515704/executions/4a554bfad780c6e3bd91fd2c937465049d             
                             e35d3d/data/steps_outputs'                                                            

                    INFO     ['distilabel.pipeline'] ⌛ The steps of the pipeline will be loaded in    ]8;id=448210;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=536382;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py#1046\1046]8;;\
                             stages:                                                                               
                              * Legend: 🚰 GeneratorStep 🌐 GlobalStep 🔄 Step                                     
                              * Stage 0:                                                                           
                                - 🚰 'load_dataset'                                                                
                                - 🔄 'text_generation'                                                             
                                - 🔄 'keep_columns'                                                                

                    INFO     ['distilabel.pipeline'] ⏳ Waiting for all the steps of stage 0 to        ]8;id=569253;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=114978;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py#1382\1382]8;;\
                             load...                                                                               

[04/14/25 10:04:34] INFO     ['distilabel.pipeline'] ⏳ Steps from stage 0 loaded: 3/3                 ]8;id=984712;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=723357;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py#1418\1418]8;;\
                              * 'load_dataset' replicas: 1/1                                                       
                              * 'text_generation' replicas: 1/1                                                    
                              * 'keep_columns' replicas: 1/1                                                       

                    INFO     ['distilabel.pipeline'] ✅ All the steps from stage 0 have been loaded!   ]8;id=739112;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=427421;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py#1422\1422]8;;\

...

                    INFO     ['distilabel.step.text_generation'] 🏁 Finished running step       ]8;id=305884;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py\step_wrapper.py]8;;\:]8;id=471195;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py#129\129]8;;\
                             'text_generation' (replica ID: 0)                                                     

                    INFO     ['distilabel.step.keep_columns'] 📦 Processing batch 0 in          ]8;id=859843;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py\step_wrapper.py]8;;\:]8;id=896142;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py#230\230]8;;\
                             'keep_columns' (replica ID: 0)                                                        

                    INFO     ['distilabel.step.keep_columns'] 📨 Step 'keep_columns' sending    ]8;id=646387;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py\step_wrapper.py]8;;\:]8;id=932746;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py#290\290]8;;\
                             batch 0 to output queue                                                               

                    INFO     ['distilabel.step.keep_columns'] 🏁 Finished running step          ]8;id=157147;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py\step_wrapper.py]8;;\:]8;id=721309;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py#129\129]8;;\
                             'keep_columns' (replica ID: 0)                                                        

Generating train split: 0 examples [00:00, ? examples/s]

In [62]:
distiset_text_generation1["default"]["train"][0]

{'instruction': 'Provide step-by-step instructions on how to make a safe and effective homemade all-purpose cleaner from common household ingredients. The guide should include measurements, tips for storing the cleaner, and additional variations or scents that can be added. Additionally, the guide should be written in clear and concise language, with helpful visuals or photographs to aid in the process.',
 'generation': 'Homemade All-Purpose Cleaner Recipe:\n\nIngredients:\n- 1 cup white vinegar\n- 1 cup water\n- 1 tablespoon baking soda\n- Essential oils (optional, for scent)\n\nInstructions:\n1. In a spray bottle, combine 1 cup of white vinegar and 1 cup of water. The vinegar helps to disinfect and clean, while water dilutes the vinegar to make it safe for use on various surfaces.\n2. Add in 1 tablespoon of baking soda to the mixture. Baking soda is a natural abrasive that helps to scrub away tough stains and grime.\n3. If desired, add a few drops'}

###Given dataset2 do text generations for instructions

In [59]:

#With dataset2
with Pipeline(name="text_generation_dataset2") as pipeline_text_generation2:
  load_data = LoadDataFromDicts(
      name="load_data",
      data=dataset2,
      output_mappings={"topic": "instruction"},
  )
  text_generation = TextGeneration(name="text_generation", llm=llm)
  keep_columns = KeepColumns(
        name="keep_columns",
        columns=[
            "instruction",
            "generation",
        ],
    )


  load_data >> text_generation >> keep_columns
distiset_text_generation2 = pipeline_text_generation2.run(use_cache=False)

[04/14/25 10:32:49] INFO     ['distilabel.pipeline'] 📝 Pipeline data will be written to               ]8;id=900996;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=100679;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py#1015\1015]8;;\
                             '/root/.cache/distilabel/pipelines/text_generation_dataset2/eeac2f0bcefc6             
                             e60d5db8942e67e5ac6d2bafe49/executions/7232845f73ea94979051fa664c8953c9b1             
                             075b55/data/steps_outputs'                                                            

                    INFO     ['distilabel.pipeline'] ⌛ The steps of the pipeline will be loaded in    ]8;id=154191;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=731563;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py#1046\1046]8;;\
                             stages:                                                                               
                              * Legend: 🚰 GeneratorStep 🌐 GlobalStep 🔄 Step                                     
                              * Stage 0:                                                                           
                                - 🚰 'load_data'                                                                   
                                - 🔄 'text_generation'                                                             
                                - 🔄 'keep_columns'                                                                

                    INFO     ['distilabel.pipeline'] ⏳ Waiting for all the steps of stage 0 to        ]8;id=370901;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=463443;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py#1382\1382]8;;\
                             load...                                                                               

[04/14/25 10:32:52] INFO     ['distilabel.pipeline'] ⏳ Steps from stage 0 loaded: 3/3                 ]8;id=715934;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=201396;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py#1418\1418]8;;\
                              * 'load_data' replicas: 1/1                                                          
                              * 'text_generation' replicas: 1/1                                                    
                              * 'keep_columns' replicas: 1/1                                                       

                    INFO     ['distilabel.pipeline'] ✅ All the steps from stage 0 have been loaded!   ]8;id=401358;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=390138;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py#1422\1422]8;;\

...

                    INFO     ['distilabel.step.text_generation'] 🏁 Finished running step       ]8;id=920960;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py\step_wrapper.py]8;;\:]8;id=813768;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py#129\129]8;;\
                             'text_generation' (replica ID: 0)                                                     

                    INFO     ['distilabel.step.keep_columns'] 📦 Processing batch 0 in          ]8;id=151429;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py\step_wrapper.py]8;;\:]8;id=187866;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py#230\230]8;;\
                             'keep_columns' (replica ID: 0)                                                        

                    INFO     ['distilabel.step.keep_columns'] 📨 Step 'keep_columns' sending    ]8;id=850696;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py\step_wrapper.py]8;;\:]8;id=961438;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py#290\290]8;;\
                             batch 0 to output queue                                                               

                    INFO     ['distilabel.step.keep_columns'] 🏁 Finished running step          ]8;id=17588;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py\step_wrapper.py]8;;\:]8;id=86087;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py#129\129]8;;\
                             'keep_columns' (replica ID: 0)                                                        

Generating train split: 0 examples [00:00, ? examples/s]

In [61]:
distiset_text_generation2["default"]["train"][:]

{'instruction': ['Algebraic Expressions',
  'Linear Equations',
  'Quadratic Equations',
  'Polynomial Functions',
  'Rational Expressions',
  'Exponential Functions',
  'Logarithmic Functions',
  'Sequences and Series',
  'Matrices',
  'Determinants',
  'Complex Numbers',
  'Trigonometry',
  'Geometry',
  'Coordinate Geometry',
  'Vector Algebra',
  'Statistics',
  'Probability',
  'Calculus',
  'Differential Calculus',
  'Integral Calculus',
  'Limits and Continuity',
  'Differentiation',
  'Integration',
  'Theorems of Calculus',
  'Mathematical Reasoning',
  'Set Theory',
  'Number Theory',
  'Permutations and Combinations',
  'Binomial Theorem',
  'Arithmetic Progressions',
  'Geometric Progressions',
  'Harmonic Progressions',
  'Trigonometric Ratios',
  'Trigonometric Identities',
  'Inverse Trigonometric Functions',
  'Hyperbolic Functions',
  'Conic Sections',
  'Circle Geometry',
  'Ellipse Geometry',
  'Parabola Geometry',
  'Hyperbola Geometry',
  'Function Theory',
  'Grap

#Improve prompts with SelfInstruct

In [65]:
from distilabel.pipeline import Pipeline
from distilabel.steps import LoadDataFromHub, KeepColumns
from distilabel.steps.tasks import SelfInstruct

with Pipeline(name="self_instruct") as pipeline_self_instruct:
    load_hub_dataset = LoadDataFromHub(
        name="load_dataset",
        repo_id="DIBT/10k_prompts_ranked",
        output_mappings={"prompt": "input"},
        split="train",
        num_examples=1
    )
    self_instruct = SelfInstruct(name="self_instruct", llm=llm, num_instructions=1)
    keep_columns = KeepColumns(
        name="keep_columns",
        columns=[
            "input",
            "instructions",
        ],
    )
    load_hub_dataset >> self_instruct >> keep_columns

distiset_self_instruct = pipeline_self_instruct.run(use_cache=False)

[04/14/25 10:43:53] INFO     ['distilabel.pipeline'] 📝 Pipeline data will be written to               ]8;id=41888;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=865916;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py#1015\1015]8;;\
                             '/root/.cache/distilabel/pipelines/self_instruct/9bef6e9f7a338aa0474fd2a1             
                             18b8ea4b52c5a5e5/executions/7dd820ae4a0304d8d35ddb06c0dd660ff58f2556/data             
                             /steps_outputs'                                                                       

                    INFO     ['distilabel.pipeline'] ⌛ The steps of the pipeline will be loaded in    ]8;id=739386;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=816382;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py#1046\1046]8;;\
                             stages:                                                                               
                              * Legend: 🚰 GeneratorStep 🌐 GlobalStep 🔄 Step                                     
                              * Stage 0:                                                                           
                                - 🚰 'load_dataset'                                                                
                                - 🔄 'self_instruct'                                                               
                                - 🔄 'keep_columns'                                                                

                    INFO     ['distilabel.pipeline'] ⏳ Waiting for all the steps of stage 0 to        ]8;id=148049;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=870720;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py#1382\1382]8;;\
                             load...                                                                               

[04/14/25 10:43:56] INFO     ['distilabel.pipeline'] ⏳ Steps from stage 0 loaded: 3/3                 ]8;id=242973;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=343467;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py#1418\1418]8;;\
                              * 'load_dataset' replicas: 1/1                                                       
                              * 'self_instruct' replicas: 1/1                                                      
                              * 'keep_columns' replicas: 1/1                                                       

                    INFO     ['distilabel.pipeline'] ✅ All the steps from stage 0 have been loaded!   ]8;id=177473;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=793733;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py#1422\1422]8;;\

...

                    INFO     ['distilabel.step.self_instruct'] 🏁 Finished running step         ]8;id=405483;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py\step_wrapper.py]8;;\:]8;id=457391;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py#129\129]8;;\
                             'self_instruct' (replica ID: 0)                                                       

                    INFO     ['distilabel.step.keep_columns'] 📦 Processing batch 0 in          ]8;id=933650;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py\step_wrapper.py]8;;\:]8;id=610019;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py#230\230]8;;\
                             'keep_columns' (replica ID: 0)                                                        

                    INFO     ['distilabel.step.keep_columns'] 📨 Step 'keep_columns' sending    ]8;id=132074;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py\step_wrapper.py]8;;\:]8;id=868592;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py#290\290]8;;\
                             batch 0 to output queue                                                               

                    INFO     ['distilabel.step.keep_columns'] 🏁 Finished running step          ]8;id=990818;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py\step_wrapper.py]8;;\:]8;id=676393;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py#129\129]8;;\
                             'keep_columns' (replica ID: 0)                                                        

Generating train split: 0 examples [00:00, ? examples/s]

In [67]:
distiset_self_instruct["default"]["train"][0]

{'input': 'Provide step-by-step instructions on how to make a safe and effective homemade all-purpose cleaner from common household ingredients. The guide should include measurements, tips for storing the cleaner, and additional variations or scents that can be added. Additionally, the guide should be written in clear and concise language, with helpful visuals or photographs to aid in the process.',
 'instructions': ['How can I customize the scent of the homemade all-purpose cleaner?']}

#Evol Quality

In [68]:
from distilabel.pipeline import Pipeline
from distilabel.steps import LoadDataFromDicts, KeepColumns
from distilabel.steps.tasks import EvolQuality

with Pipeline(name="evol_quality") as pipeline_evol_quality:
    load_data = LoadDataFromDicts(
        name="load_data",
        data=[
            {
                'instruction': "What is the capital of India?",
                'response': "New Delhi is the capital of India",
            }
        ],
    )
    evol_quality = EvolQuality(name="evol_quality", llm=llm, num_evolutions=1)
    keep_columns = KeepColumns(
        name="keep_columns",
        columns=[
            "instruction",
            "response",
            "evolved_response",
        ],
    )
    load_data >> evol_quality >> keep_columns

distiset_evol_quality = pipeline_evol_quality.run(use_cache=False)

[04/14/25 10:51:22] INFO     ['distilabel.pipeline'] 📝 Pipeline data will be written to               ]8;id=290651;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=545718;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py#1015\1015]8;;\
                             '/root/.cache/distilabel/pipelines/evol_quality/d3672fb9e1f001a6f7697f180             
                             880a8055964d843/executions/df225681683ba335e581363055d64f63d6afb29c/data/             
                             steps_outputs'                                                                        

                    INFO     ['distilabel.pipeline'] ⌛ The steps of the pipeline will be loaded in    ]8;id=472967;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=225388;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py#1046\1046]8;;\
                             stages:                                                                               
                              * Legend: 🚰 GeneratorStep 🌐 GlobalStep 🔄 Step                                     
                              * Stage 0:                                                                           
                                - 🚰 'load_data'                                                                   
                                - 🔄 'evol_quality'                                                                
                                - 🔄 'keep_columns'                                                                

                    INFO     ['distilabel.pipeline'] ⏳ Waiting for all the steps of stage 0 to        ]8;id=652956;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=483874;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py#1382\1382]8;;\
                             load...                                                                               

[04/14/25 10:51:25] INFO     ['distilabel.pipeline'] ⏳ Steps from stage 0 loaded: 3/3                 ]8;id=155145;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=434817;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py#1418\1418]8;;\
                              * 'load_data' replicas: 1/1                                                          
                              * 'evol_quality' replicas: 1/1                                                       
                              * 'keep_columns' replicas: 1/1                                                       

                    INFO     ['distilabel.pipeline'] ✅ All the steps from stage 0 have been loaded!   ]8;id=519460;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=756437;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py#1422\1422]8;;\

...

                    INFO     ['distilabel.step.evol_quality'] 🏁 Finished running step          ]8;id=356348;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py\step_wrapper.py]8;;\:]8;id=88997;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py#129\129]8;;\
                             'evol_quality' (replica ID: 0)                                                        

[04/14/25 10:51:26] INFO     ['distilabel.step.keep_columns'] 📦 Processing batch 0 in          ]8;id=771593;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py\step_wrapper.py]8;;\:]8;id=797304;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py#230\230]8;;\
                             'keep_columns' (replica ID: 0)                                                        

                    INFO     ['distilabel.step.keep_columns'] 📨 Step 'keep_columns' sending    ]8;id=96016;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py\step_wrapper.py]8;;\:]8;id=218137;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py#290\290]8;;\
                             batch 0 to output queue                                                               

                    INFO     ['distilabel.step.keep_columns'] 🏁 Finished running step          ]8;id=697509;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py\step_wrapper.py]8;;\:]8;id=29378;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py#129\129]8;;\
                             'keep_columns' (replica ID: 0)                                                        

Generating train split: 0 examples [00:00, ? examples/s]

In [69]:
distiset_evol_quality["default"]["train"][0]

{'instruction': 'What is the capital of India?',
 'response': 'New Delhi is the capital of India',
 'evolved_response': 'The capital of India is New Delhi, known for its rich history and vibrant culture.'}